In [28]:
import requests
import json
import csv

# NBA Feed API URL Endpoints

In [29]:
base_url = "https://data.nba.com/data/v2022/json/mobile_teams/nba"

base_url_2022 = base_url + "/2022"
# Teams
teams_info_url = base_url_2022 + "/teams/00_team_info.json"
# Players
players_info_url = base_url_2022 + "/players/00_player_info.json"
# Games
games_info_url = base_url_2022 + "/league/00_full_schedule.json"
# Game Lineup
def get_game_lineup_url(game_id: str):
    return base_url_2022 + f'''/scores/roster_lineup/{game_id}_roster_lineup.json'''
# Team Roster
def get_team_schedule_url(team_name: str):
    return base_url_2022 + f'''/teams/{team_name}_roster.json'''
# Box Score For Game
def get_box_score_for_game_url(game_id: str):
    return base_url_2022 + f'''/scores/gamedetail/{game_id}_gamedetail.json'''
# Team Games Schedule, not necessarily will be needed.
def get_team_schedule_url(team_name: str):
    return base_url_2022 + f'''/teams/{team_name}_schedule.json'''

In [30]:
# Request according to the url
def request_to_nba_feed(url: str):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}') 
    except requests.exceptions.ConnectionError as conn_err:
        print(f'Error Connecting: {conn_err}') 
    except requests.exceptions.Timeout as time_err:
        print(f'Timeout Error: {time_err}') 
    except requests.exceptions.RequestException as req_err:
        print(f'Something went wrong: {req_err}') 


# Utils

In [31]:
def get_percentage(numerator, denominator) -> int:
    if(denominator == 0 or  numerator == 0):
        return 0
    else:
        return int((numerator / denominator) * 100)

In [32]:
def save_into_csv(file_name, elements):
    # Get the headers (property names)
    headers = [prop for prop in dir(elements[0]) if not callable(getattr(elements[0], prop)) and not prop.startswith("__")]

    with open(f'{file_name}.csv', "w", newline="") as f:
        csv_writer = csv.writer(f)
        # Write headers
        csv_writer.writerow(headers)
        # Iterate and save into csv
        for element in elements:
            csv_writer.writerow([getattr(element, prop) for prop in headers])

# Class objects:

In [33]:
class Team:
    def __init__(self, ID, TeamName, TeamCode, City):
        self.ID = ID
        self.TeamName = TeamName
        self.TeamCode = TeamCode
        self.City = City  
        
    def __str__(self):
        return f'{self.ID} {self.TeamName} {self.TeamCode} {self.City}'

In [34]:
class Player:
    def __init__(self, ID, FirstName, LastName, DateOfBirth, Position, Height, Weight, YearsInLeague, Country):
        self.ID = ID
        self.FirstName = FirstName
        self.LastName = LastName
        self.DateOfBirth = DateOfBirth
        self.Position = Position
        self.Height = Height
        self.Weight = Weight
        self.YearsInLeague = YearsInLeague
        self.Country = Country
        
    def __str__(self):
        return f"ID: {self.ID}, Name: {self.FirstName} {self.LastName}, DOB: {self.DateOfBirth}, Position: {self.Position}, Height: {self.Height}, Weight: {self.Weight}, Years in League: {self.YearsInLeague}, Country: {self.Country}"

In [35]:
class TeamRoster:
    def __init__(self, TeamID, PlayerID):
        self.TeamID = TeamID
        self.PlayerID = PlayerID
        
    def __str__(self):
        return f'{self.TeamID} {self.PlayerID}'

In [36]:
class Game:
    def __init__(self, ID, Date, HomeTeamID, VisitorTeamID, LiveStatus):
        self.ID = ID
        self.Date = Date
        self.HomeTeamID = HomeTeamID
        self.VisitorTeamID = VisitorTeamID
        self.LiveStatus = LiveStatus
        
    def __str__(self):
        return f'{self.ID} {self.Date} {self.HomeTeamID} {self.VisitorTeamID} {self.LiveStatus}'

In [37]:
class GameLineUp:    
    def __init__(self, GameID, TeamID, PlayerID):
        self.GameID = GameID
        self.TeamID = TeamID
        self.PlayerID = PlayerID
        
    def __str__(self):
        return f'{self.GameID} {self.TeamID} {self.PlayerID}'

In [38]:
class GameScore:
    def __init__(self, GameID, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TO, PTS):
        self.GameID = GameID
        self.FGA = FGA
        self.FGM = FGM
        self.FGPER = FGPER
        self.ThreePA = ThreePA
        self.ThreePM = ThreePM
        self.ThreePPER = ThreePPER
        self.FTA = FTA
        self.FTM = FTM
        self.FTPER = FTPER
        self.OREB = OREB
        self.DREB = DREB
        self.AST = AST
        self.STL = STL
        self.BLK = BLK
        self.TO = TO
        self.PTS = PTS
        
    def __str__(self):
        return f'GameID: {self.GameID}, FGA: {self.FGA}, FGM: {self.FGM}, FGPER: {self.FGPER}, ThreePM: {self.ThreePM}, ThreePA: {self.ThreePA}, ThreePPER: {self.ThreePPER}, FTM: {self.FTM}, FTA: {self.FTA}, FTPER: {self.FTPER}, OREB: {self.OREB}, DREB: {self.DREB}, AST: {self.AST}, STL: {self.STL}, BLK: {self.BLK}, TO: {self.TO}, PTS: {self.PTS}'

In [39]:
class GameTeamScore(GameScore):
    def __init__(self, GameID, TeamID, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TO, PTS):
        self.TeamID = TeamID
        super().__init__(GameID, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TO, PTS)

    def __str__(self):
        return f'''TeamID: {self.TeamID}, {super().__str__()}'''

In [40]:
class GamePlayerScore(GameScore):
    def __init__(self, GameID, PlayerID, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TO, PTS):
        self.PlayerID = PlayerID
        super().__init__(GameID, FGA, FGM, FGPER, ThreePA, ThreePM, ThreePPER, FTA, FTM, FTPER, OREB, DREB, AST, STL, BLK, TO, PTS)
        

    def __str__(self):
        return f'''PlayerID: {self.PlayerID}, {super().__str__()}'''

# Parsers

In [41]:
# Team Parser
def team_parser(raw_data) -> list[Team]:

    # Access the 't' list within the json
    teams_data = raw_data['data']['tms']['t']

    # Initialize an empty list to hold the teams
    teams_list = []

    # Iterate through each team in the teams_data list
    for team in teams_data:
        # Extract the relevant information from the json and create a new Teams object
        id = team['tid']
        name = team['tn']
        code = team['ta']
        city = team['tc']
        teams_list.append(Team(id, name, code, city))

    return teams_list

In [42]:
# Player Parser
def player_parser(raw_data) -> list[Player]:
    
    # Access the 'pl' list within the json
    players_data = raw_data['data']['pls']['pl']

    # Initialize an empty list to hold the players
    players_list = []

    # Iterate through each player in the players_data list
    for player in players_data:
        # Extract the relevant information from the json and create a new Players object
        id = player['pid']
        first_name = player['fn']
        last_name = player['ln']
        date_of_birth = player['dob']
        position = player['pos']
        height =  player['ht']
        weight =  player['wt']
        years_in_league =  player['y']
        country =  player['co']
        players_list.append(Player(id, first_name, last_name, date_of_birth, position, height, weight, years_in_league, country))
    
    return players_list

In [43]:
# Team Roster Parser
def team_roster_parser(raw_data) -> list[TeamRoster]:
    
    # Access the 'pl' list within the json
    players_data = raw_data['data']['pls']['pl']

    # Initialize an empty list to hold the players
    team_roster_list = []

    # Iterate through each player in the players_data list
    for player in players_data:
        # Extract the relevant information from the json and create a new TeamRoster object
        team_id = player['tid']
        player_id = player['pid']
        team_roster_list.append(TeamRoster(team_id, player_id))

    return team_roster_list

In [44]:
# Game Parser
def game_parser(raw_data) -> list[Game]:
    
    # Access the 'lscd' list within the json
    games_data = raw_data['data']['lscd']

    # Initialize an empty list to hold the players
    games_list = []

    # Iterate through each month in the games_data list
    for games_month in games_data:
        game_data_raw = games_month['mscd']['g']
        for game in game_data_raw:
            # Extract the relevant information from the json and create a new Game object
            id = game['gid']
            date = game['gdtutc']
            home_team_id = game['h']['tid']
            visitors_team_id = game['v']['tid']
            live_status = game['stt']
            games_list.append(Game(id, date, home_team_id, visitors_team_id, live_status))

    return games_list

In [45]:
# Game LineUp Parser
def game_lineup_parser(raw_data) -> list[GameLineUp]:
    
    # Access the 'data' list within the json
    lineups_teams_data = raw_data['data']
    #Extract game_id
    game_id = lineups_teams_data['gameId']
    
    # Initialize an empty list to hold the players
    game_lineup_list = []
    
    #Extract only the home team
    home_players_lineups = lineups_teams_data['home']
    team_lineup_id = home_players_lineups['teamId']
    #Iterate only on the players
    for home_player_lineup in home_players_lineups['players']:
        team_id = team_lineup_id
        player_id = home_player_lineup['personId']
        game_lineup_list.append(GameLineUp(game_id, team_id, player_id))
        
    #Extract only the visitor team
    visitor_players_lineups = lineups_teams_data['visitor']
    team_lineup_id = visitor_players_lineups['teamId']
    #Iterate only on the players
    for home_player_lineup in visitor_players_lineups['players']:
        # Extract the relevant information from the json and create a new TeamRoster object
        team_id = team_lineup_id
        player_id = home_player_lineup['personId']
        game_lineup_list.append(GameLineUp(game_id, team_id, player_id))
    
    return game_lineup_list

In [46]:
# Game Box Score Parser
def game_boxscore_parser(raw_data) -> (list[GameTeamScore], list[GamePlayerScore]):
    
    # Access the 'data' list within the json
    boxscore_data = raw_data['data']['g']
    # Extract game_id
    game_id = boxscore_data['gid']
    
    # Initialize an empty lists to hold the scores
    game_teams_score_list = []
    game_players_score_list = []
    
    # Extract visitor team score
    visitor_scores = boxscore_data['vls']
    # called the team_score_helper Func to parse the relevent fields
    game_teams_score_list.append(game_team_score_parser_helper(visitor_scores, game_id))
    
    # Iterate over the players stats and extract the relevent stat with the game_player_score Func
    for player_stats in visitor_scores['pstsg']:
        game_players_score_list.append(game_player_score_parser_helper(player_stats, game_id))
    
    # Extract Home team score
    home_scores = boxscore_data['hls']
    # called the team_score_helper Func to parse the relevent fields
    game_teams_score_list.append(game_team_score_parser_helper(home_scores, game_id))
    
    # Iterate over the players stats and extract the relevent stat with the game_player_score Func
    for player_stats in home_scores['pstsg']:
        game_players_score_list.append(game_player_score_parser_helper(player_stats, game_id))

    return (game_teams_score_list, game_players_score_list)

# Game Team Score Parser helper
def game_team_score_parser_helper(team_score_json, game_id) -> GameTeamScore:
    team_points = team_score_json['s']
    team_id = team_score_json['tid'] 
    
    team_stat = team_score_json['tstsg']
    fga = team_stat['fga']
    fgm = team_stat['fgm']   
    fg_per = get_percentage(team_stat['fgm'], team_stat['fga'])
    three_pa = team_stat['tpa']
    three_pm = team_stat['tpm']
    three_per = get_percentage(team_stat['tpm'], team_stat['tpa'])
    fta = team_stat['fta']
    ftm = team_stat['ftm']
    ft_per = get_percentage(team_stat['ftm'], team_stat['fta'])
    o_reb = team_stat['oreb']
    d_reb = team_stat['dreb']
    ast = team_stat['ast']
    stl = team_stat['stl']
    blk = team_stat['blk']
    to = team_stat['tov']
    pts = team_points    
    
    return GameTeamScore(game_id, team_id, fga, fgm, fg_per, three_pa, three_pm, three_per, fta, ftm, ft_per, o_reb, d_reb, ast, stl, blk, to, pts)

# Game Player Score Parser helper
def game_player_score_parser_helper(player_stat, game_id) -> GamePlayerScore:
    
    fga = player_stat['fga']
    fgm = player_stat['fgm']   
    fg_per = get_percentage(player_stat['fgm'], player_stat['fga'])
    three_pa = player_stat['tpa']
    three_pm = player_stat['tpm']
    three_per = get_percentage(player_stat['tpm'], player_stat['tpa'])
    fta = player_stat['fta']
    ftm = player_stat['ftm']
    ft_per = get_percentage(player_stat['ftm'], player_stat['fta'])
    o_reb = player_stat['oreb']
    d_reb = player_stat['dreb']
    ast = player_stat['ast']
    stl = player_stat['stl']
    blk = player_stat['blk']
    to = player_stat['tov']
    pts = player_stat['pts']
    player_id = player_stat['pid']
    
    return GamePlayerScore(game_id, player_id, fga, fgm, fg_per, three_pa, three_pm, three_per, fta, ftm, ft_per, o_reb, d_reb, ast, stl, blk, to, pts)

# Initialization

In [47]:
#Initialize Teams
all_teams = team_parser(request_to_nba_feed(teams_info_url))

In [48]:
#Initialize Players
all_players = player_parser(request_to_nba_feed(players_info_url))

In [49]:
#Initialize Games
all_games = game_parser(request_to_nba_feed(games_info_url))

In [50]:
#Initialize Rosters
all_team_rosters = team_roster_parser(request_to_nba_feed(players_info_url))

In [51]:
#game example
game_id = '0012200005'

In [52]:
game_lineup = game_lineup_parser(request_to_nba_feed(get_game_lineup_url(game_id)))

In [53]:
game_team_scores, game_player_scores = game_boxscore_parser(request_to_nba_feed(get_box_score_for_game_url(game_id)))

In [54]:
#Initialize Lineups and scores(Teams and Players) for the the 'Finish' games
all_game_lineups = []
all_game_team_scores = []
all_game_player_scores = []

final_games_ids = [game.ID for game in all_games if game.LiveStatus == 'Final']

num_of_iterations = len(final_games_ids)
counter = 0
for final_game_id in final_games_ids:
    counter += 1
    print(f'{counter} from {num_of_iterations}')
    all_game_lineups.extend(game_lineup_parser(request_to_nba_feed(get_game_lineup_url(final_game_id))))
    
    game_team_scores, game_player_scores = game_boxscore_parser(request_to_nba_feed(get_box_score_for_game_url(final_game_id)))
    all_game_team_scores.extend(game_team_scores)
    all_game_player_scores.extend(game_player_scores)

1 from 725
2 from 725
3 from 725
4 from 725
5 from 725
6 from 725
7 from 725
8 from 725
9 from 725
10 from 725
11 from 725
12 from 725
13 from 725
14 from 725
15 from 725
16 from 725
17 from 725
18 from 725
19 from 725
20 from 725
21 from 725
22 from 725
23 from 725
24 from 725
25 from 725
26 from 725
27 from 725
28 from 725
29 from 725
30 from 725
31 from 725
32 from 725
33 from 725
34 from 725
35 from 725
36 from 725
37 from 725
38 from 725
39 from 725
40 from 725
41 from 725
42 from 725
43 from 725
44 from 725
45 from 725
46 from 725
47 from 725
48 from 725
49 from 725
50 from 725
51 from 725
52 from 725
53 from 725
54 from 725
55 from 725
56 from 725
57 from 725
58 from 725
59 from 725
60 from 725
61 from 725
62 from 725
63 from 725
64 from 725
65 from 725
66 from 725
67 from 725
68 from 725
69 from 725
70 from 725
71 from 725
72 from 725
73 from 725
74 from 725
75 from 725
76 from 725
77 from 725
78 from 725
79 from 725
80 from 725
81 from 725
82 from 725
83 from 725
84 from 725
8

# Save to csv files

In [55]:
save_into_csv('Teams', all_teams)
save_into_csv('Players', all_players)
save_into_csv('Games', all_games)
save_into_csv('TeamRosters', all_team_rosters)

save_into_csv('GameLineUp', all_game_lineups)
save_into_csv('GameTeamScore', all_game_team_scores)
save_into_csv('GamePlayerScore', all_game_player_scores)